## DataFrame
- Schema(컬럼과 타입)와 선언적 언어 사용
- 이름이 지정된 열로 구성된 Dataset
- 개념적으로는 관계형 데이터베이스의 테이블과 동등
- 구조화된 파일(JSON, CSV, Parquet), Hive 테이블, 외부 데이터베이스 또는 기존 RDD와 같은 소스에서 생성 가능
- 구조화된 데이터에 대해 매우 효율적 (CPU와 메모리 효율성이 높다.)

## RDD vs. DF
- any othe type vs structed data(구조적 데이터)
- complex things(python) vs simple things(SQL)
- How(어떻게 처리 할 것인가) vs What(무엇을 처리 할 것인가)
- [ SparkContext + SQLContext ] vs [ SparkSession ]

## DF Operation
- DF standalone operations : transformation(select, where, filter, groupBy, having, join ...), action(show)
- Spark SQL
- UDF (User Define Function)

* CPU와 메모리 향상을 위해 Spark SQL과 UDF를 사용. DF standalone operation은 최적화가 없다.
* Catalyst query optimization(룰 기반, 비용기반 옵티마이저), Project tungsten(CPU, 메모리 관리 기술. Cache-awareness Computation. 병렬 처리를 위한 최신 컴파일러)를 통해 쿼리를 병렬으로 실행하고 생산성 향상
* Cache-awareness Computation : https://younggyuchun.wordpress.com/2017/01/31/spark-%ec%84%b1%eb%8a%a5%ec%9d%98-%ed%95%b5%ec%8b%ac-project-tungsten-%ed%86%ba%ec%95%84%eb%b3%b4%ea%b8%b0/

# DF(DataFrame)으로 작업하기

In [1]:
# pyspark 연결하기
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

23/11/28 09:35:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/28 09:35:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/11/28 09:35:46 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
# Create/Load DF: 스파크는 옵션을 주어 자동적으로 파일을 스캔하고 데이터 셋의 스키마를 추론하게 할 수 있다.
# data source: https://www.kaggle.com/thec03u5/fifa-18-demo-player-dataset
df1 = spark.read.format("csv").load("./data/CompleteDataset.csv", inferSchema=True, header=True)

In [3]:
# 데이터 보여주기 (기본적으로 20개의 데이터만 보여준다. int 타입의 인자를 보여주는 행의 개수를 지정할 수 있다.)
df1.show(5)

23/11/28 09:36:27 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/11/28 09:36:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Name, Age, Photo, Nationality, Flag, Overall, Potential, Club, Club Logo, Value, Wage, Special, Acceleration, Aggression, Agility, Balance, Ball control, Composure, Crossing, Curve, Dribbling, Finishing, Free kick accuracy, GK diving, GK handling, GK kicking, GK positioning, GK reflexes, Heading accuracy, Interceptions, Jumping, Long passing, Long shots, Marking, Penalties, Positioning, Reactions, Short passing, Shot power, Sliding tackle, Sprint speed, Stamina, Standing tackle, Strength, Vision, Volleys, CAM, CB, CDM, CF, CM, ID, LAM, LB, LCB, LCM, LDM, LF, LM, LS, LW, LWB, Preferred Positions, RAM, RB, RCB, RCM, RDM, RF, RM, RS, RW, RWB, ST
 Schema: _c0, Name, Age, Photo, Nationality, Flag, Overall, Potential, Clu

+---+-----------------+---+--------------------+-----------+--------------------+-------+---------+-------------------+--------------------+------+-----+-------+------------+----------+-------+-------+------------+---------+--------+-----+---------+---------+------------------+---------+-----------+----------+--------------+-----------+----------------+-------------+-------+------------+----------+-------+---------+-----------+---------+-------------+----------+--------------+------------+-------+---------------+--------+------+-------+----+----+----+----+----+------+----+----+----+----+----+----+----+----+----+----+-------------------+----+----+----+----+----+----+----+----+----+----+----+
|_c0|             Name|Age|               Photo|Nationality|                Flag|Overall|Potential|               Club|           Club Logo| Value| Wage|Special|Acceleration|Aggression|Agility|Balance|Ball control|Composure|Crossing|Curve|Dribbling|Finishing|Free kick accuracy|GK diving|GK handling|

In [4]:
# How many partitions in DF?
df1.rdd.getNumPartitions()

2

In [5]:
# Increase/Decrease the partitions in DF (파티션 수 늘리거나 줄이기)
df2 = df1.repartition(4) # 파티션 늘리기
print(df2.rdd.getNumPartitions())

df3 = df2.coalesce(2) # 파티션 줄이기
print(df3.rdd.getNumPartitions())

4
2


In [6]:
# Rename Columns and Amend NULLs (이름 변경하고 NULL 값 채우기)
df2 = df2.withColumnRenamed("_c0", "RowID") \
    .withColumnRenamed("Ball control", "Ball_Control") \
    .withColumnRenamed("Sliding tackle", "Sliding_Tackle")

df2.na.fill({"RAM": 10, "RB": 1}).show(3)

23/11/28 09:36:39 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Name, Age, Photo, Nationality, Flag, Overall, Potential, Club, Club Logo, Value, Wage, Special, Acceleration, Aggression, Agility, Balance, Ball control, Composure, Crossing, Curve, Dribbling, Finishing, Free kick accuracy, GK diving, GK handling, GK kicking, GK positioning, GK reflexes, Heading accuracy, Interceptions, Jumping, Long passing, Long shots, Marking, Penalties, Positioning, Reactions, Short passing, Shot power, Sliding tackle, Sprint speed, Stamina, Standing tackle, Strength, Vision, Volleys, CAM, CB, CDM, CF, CM, ID, LAM, LB, LCB, LCM, LDM, LF, LM, LS, LW, LWB, Preferred Positions, RAM, RB, RCB, RCM, RDM, RF, RM, RS, RW, RWB, ST
 Schema: _c0, Name, Age, Photo, Nationality, Flag, Overall, Potential, Club, Club Logo, Value, Wage, Special, Acceleration, Aggression, Agility, Balance, Ball control, Composure, Crossing, Curve, Dribbling, Finishing, Free kick accuracy, GK diving, GK ha

+-----+------------+---+--------------------+-----------+--------------------+-------+---------+--------------------+--------------------+-----+----+-------+------------+----------+-------+-------+------------+---------+--------+-----+---------+---------+------------------+---------+-----------+----------+--------------+-----------+----------------+-------------+-------+------------+----------+-------+---------+-----------+---------+-------------+----------+--------------+------------+-------+---------------+--------+------+-------+----+----+----+----+----+------+----+----+----+----+----+----+----+----+----+----+-------------------+----+----+----+----+----+----+----+----+----+----+----+
|RowID|        Name|Age|               Photo|Nationality|                Flag|Overall|Potential|                Club|           Club Logo|Value|Wage|Special|Acceleration|Aggression|Agility|Balance|Ball_Control|Composure|Crossing|Curve|Dribbling|Finishing|Free kick accuracy|GK diving|GK handling|GK kicki

In [ ]:
# Transformation (SELECT): 원하는 컬럼만 추출된 데이터프레임을 반환. 
df2.select("Name", "Overall").distinct().show() # distinct()를 통해 [Name, Overal]이 중복인 데이터 제거

In [ ]:
# Transformation (FILTER): 지정한 열이 특정 조건을 만족하는 새로운 데이터프레임을 반환
df2.filter(df["Overall"] > 70).show()

In [ ]:
# SELCT로 필요한 열만 추출 후 FILTER로 데이터에 대한 조건식을 만족하는 데이터만 추출
df2.select("Overall", "Name", "Age").filter(df2["Overall"] > 70).show()

In [ ]:
# FILTER 대신 WHERE 으로 대체 가능
df2.select("Overall", "Name", "Age").where(df2["Overall"] > 70).show()

In [ ]:
# 여러 Transformation을 같이 써서 응용하기
df2.where[df2["Overall"] > 70] \ # 조건식 적용
    .groupBy("Age") \ # groupBy (집계 함수의 기준 컬럼)
    .count() \ # count (집계 함수)
    .sort("Age") \ # sort (정렬 함수)
    .show() # action (출력 함수)